# CALCUL DE LA PERTINENCE TOTALE D'UN PROSPECT 

In [74]:
# Cacul de la pertinence de l'offre 1 et 6 : 1-2= -1 
# Peu importe le choix des offres choisies il faut ajouter une pertinence de -1 au calcul total de la pertinence

 ![](https://i.imgur.com/vZzyTTy.png)

![](https://i.imgur.com/ycWhkKr.png)

In [75]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['instant-gratification', 'pertinence1', 'pertinence']


In [76]:
# Créationd ddu répertoire pertinence (directory)
print(os.listdir("../input/pertinence"))

['pertinence.csv']


In [77]:
#Importation de la table que nous avons crée sous format csv 
import pandas as pd 
data=pd.read_csv("../input/pertinence/pertinence.csv")
data

,Offre,Pertinence
0,1,1
1,2,0
2,3,-1
3,4,3
4,5,-1
5,6,-2


In [78]:
# Transformation de la colonne Offre en index.
data.set_index("Offre")

,Pertinence
Offre,
1,1
2,0
3,-1
4,3
5,-1
6,-2


In [79]:
# Somme de la colonne Pertinence si le prospect accède à l'ensemble des 6 offres vaut 0
data["Pertinence"].sum()

0

In [80]:
# Sur les 6 offres le prospect en choisit obligatoirement au minimum 2 : la première offre et la dernière offre (ici respectivement 1 et 6)
# Le prospect aura au minimum 2 offres et au maximum 6 offres: [2;6]
# On va donc raisonner en termes de nombres d'offres prises, s'il choisit une offre supplementaire : cela fera 2+1=3 donc 3 offres et il a sauté 3 
# offres
# hormis l'offre n°1 et l'offre n°6, il peut donc choisir des offres supplémentaires du lot [2,3,4,5] 

longueur=len(data)+1
List1=list(range(longueur))
List1=List1[2:-1]
List1

[2, 3, 4, 5]

In [81]:
# Voici les deux offres obligatoires n°1 et n°6
ee=list(range(longueur))
one_six=[ee[1],ee[-1]]
one_six


[1, 6]

In [82]:
# La Pertinence Totale des offres obligatoire n°1 et n°6 vaut -1.
dern=data[data["Offre"].isin(one_six)][["Pertinence"]].sum()
dern

Pertinence   -1
dtype: int64

# I. EXEMPLE AVEC 2 OFFRES SELECTIONNEES

![](https://i.imgur.com/kz0x07a.png)

![](https://i.imgur.com/7ORiXml.png)

In [83]:
# On calcule la pertinence pour les combinaisons possible de 2
from itertools import combinations 
  

comb = combinations(List1, 2) 
  
# Print the obtained combinations 
for i in list(comb): 
    A=i
    B=data[data["Offre"].isin(i)][["Pertinence"]].sum()
    
    print(A,B)





(2, 3) Pertinence   -1
dtype: int64
(2, 4) Pertinence    3
dtype: int64
(2, 5) Pertinence   -1
dtype: int64
(3, 4) Pertinence    2
dtype: int64
(3, 5) Pertinence   -2
dtype: int64
(4, 5) Pertinence    2
dtype: int64


In [84]:
#Ajoutons y la pertinence total des offres n°1 et n°6 que nous avons calculé plus tôt 
from itertools import combinations 
  

comb = combinations(List1, 2) 
  
# Print the obtained combinations 
for i in list(comb): 
    A=i
    B=data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern
    
    print(A,B)



(2, 3) Pertinence   -2
dtype: int64
(2, 4) Pertinence    2
dtype: int64
(2, 5) Pertinence   -2
dtype: int64
(3, 4) Pertinence    1
dtype: int64
(3, 5) Pertinence   -3
dtype: int64
(4, 5) Pertinence    1
dtype: int64


In [85]:
from itertools import combinations 
comb = combinations(List1, 2) 
lst=[max(data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern) for i in list(comb)]
print("Pour la selection de 4 offres (les deux obligatoires + deux choisies, on a les pertinences et combinaisons suivantes :")
print("pertinence :",lst)
comb = combinations(List1, 2) 
lst1=[i for i in list(comb)]
print("numero_offre :",lst1)


Pour la selection de 4 offres (les deux obligatoires + deux choisies, on a les pertinences et combinaisons suivantes :
pertinence : [-2, 2, -2, 1, -3, 1]
numero_offre : [(2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


In [86]:
# Création de la DataFrame
DF= pd.DataFrame(
    {'pertinence': lst,
     'numero_offre': lst1
    })
DF['numero_offre'] = '(1,6)+' + DF['numero_offre'].astype(str)
DF

,pertinence,numero_offre
0,-2,"(1,6)+(2, 3)"
1,2,"(1,6)+(2, 4)"
2,-2,"(1,6)+(2, 5)"
3,1,"(1,6)+(3, 4)"
4,-3,"(1,6)+(3, 5)"
5,1,"(1,6)+(4, 5)"


In [87]:
# Selection de la pertinence maximale parmis les 6 combinaisons possibles
# La pertinence maximal est de 2 et elle est atteinte lorsque le prospect choisit les offres 1,6,2 et 4
DF[DF["pertinence"]==DF["pertinence"].max()]

,pertinence,numero_offre
1,2,"(1,6)+(2, 4)"


# II. Automatisation du programme pour calculer l'ensemble des combinaisons possibles de la liste d'offres possibles

In [102]:
# Calcul de toutes les combinaisons possible et des pertinences totales par combinaison(la pertinence des offres n°1 et n°6 est ajouté dans A avec le
# paramètre "dern")
import itertools 

for L in range(0,len(List1)+1):
    for subset in itertools.combinations(List1,L):
        print(subset)
        A=data[data["Offre"].isin(subset)][["Pertinence"]].sum()+dern
        print(A)
        

()
Pertinence   -1
dtype: int64
(2,)
Pertinence   -1
dtype: int64
(3,)
Pertinence   -2
dtype: int64
(4,)
Pertinence    2
dtype: int64
(5,)
Pertinence   -2
dtype: int64
(2, 3)
Pertinence   -2
dtype: int64
(2, 4)
Pertinence    2
dtype: int64
(2, 5)
Pertinence   -2
dtype: int64
(3, 4)
Pertinence    1
dtype: int64
(3, 5)
Pertinence   -3
dtype: int64
(4, 5)
Pertinence    1
dtype: int64
(2, 3, 4)
Pertinence    1
dtype: int64
(2, 3, 5)
Pertinence   -3
dtype: int64
(2, 4, 5)
Pertinence    1
dtype: int64
(3, 4, 5)
Pertinence    0
dtype: int64
(2, 3, 4, 5)
Pertinence    0
dtype: int64


In [103]:


lst=[max(data[data["Offre"].isin(subset)][["Pertinence"]].sum()+dern) for L in range(0,len(List1)+1) for subset in itertools.combinations(List1,L)]
lst







[-1, -1, -2, 2, -2, -2, 2, -2, 1, -3, 1, 1, -3, 1, 0, 0]

In [104]:

lst1=[subset for L in range(0,len(List1)+1) for subset in itertools.combinations(List1,L)]
lst1

[(),
 (2,),
 (3,),
 (4,),
 (5,),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5),
 (2, 3, 4, 5)]

In [105]:
import itertools 

for L in range(0,len(List1)+1):
    for subset in itertools.combinations(List1,L):
        print(subset)
        

()
(2,)
(3,)
(4,)
(5,)
(2, 3)
(2, 4)
(2, 5)
(3, 4)
(3, 5)
(4, 5)
(2, 3, 4)
(2, 3, 5)
(2, 4, 5)
(3, 4, 5)
(2, 3, 4, 5)


In [106]:
DF= pd.DataFrame(
    {'col1': lst,
     'col2': lst1
    })
DF


,col1,col2
0,-1,()
1,-1,"(2,)"
2,-2,"(3,)"
3,2,"(4,)"
4,-2,"(5,)"
5,-2,"(2, 3)"
6,2,"(2, 4)"
7,-2,"(2, 5)"
8,1,"(3, 4)"
9,-3,"(3, 5)"


In [107]:
# Ajout des offres obligatoires n°1 et n°6 
DF['col2'] = '(1,6)+' + DF['col2'].astype(str)
DF

,col1,col2
0,-1,"(1,6)+()"
1,-1,"(1,6)+(2,)"
2,-2,"(1,6)+(3,)"
3,2,"(1,6)+(4,)"
4,-2,"(1,6)+(5,)"
5,-2,"(1,6)+(2, 3)"
6,2,"(1,6)+(2, 4)"
7,-2,"(1,6)+(2, 5)"
8,1,"(1,6)+(3, 4)"
9,-3,"(1,6)+(3, 5)"


In [97]:
DF[DF["col1"]==DF["col1"].max()]

,col1,col2
3,2,"(1,6)+(4,)"
6,2,"(1,6)+(2, 4)"


![](https://i.imgur.com/mLSUeT8.png)

## Dans l'exemple précédent nous avons travaillé sur 6 offres mais l'algorithme que nous avons dévellopé nous permet de réaliser la même étude pour un nombre d'offres plus important.
